In [0]:
!pip install -q six numpy scipy keras
!pip install -q git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
import keras
from keras_contrib.applications import DenseNet

In [0]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.models import load_model
from keras_contrib.callbacks import CyclicLR
from keras.callbacks import ModelCheckpoint
import os,sys

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
def modified_model(src_model,copy_weights=True,n_layers_to_pop=1):
  src_model_name = src_model.name if hasattr(src_model,'name') else 'src_model'
  print('Cloning the model '+src_model_name)
  input_model = keras.models.clone_model(src_model)
  print('Popping the last layer')
  for i in range(n_layers_to_pop):
    input_model.layers.pop()
  input_model_output = input_model.layers[-1].output
  fc1 = Dense(1024,activation='linear')(input_model_output)
  fc2=Dense(10,activation='softmax')(fc1)
  print('Creating new model by adding a penultimate Dense layer')
  modified_model = Model(input_model.inputs,fc2,name='Modified '+src_model_name)
  print('Copying weights from ',src_model_name,'to',modified_model.name)
  if copy_weights:
    #if 1<0:
    #  modified_model.set_weights(src_model.get_weights())
    #else:
    for i in range(len(input_model.layers)):
        print('\rCopying weights of layer:',i+1,end='')
        try:
          modified_model.layers[i].set_weights(input_model.layers[i].get_weights())
        except Exception as e:
          print('\nError copying weights at layer ',i+1)
    #modified_model.layers[-1].set_weights(src_model.layers[-1].get_weights())
  return modified_model

In [0]:
def add_penultimaye_layer(src_model,layer_dict,n_layers_to_pop=1,copy_weights=True): #layer_dict {type,kernels,activation}
  src_model_name = src_model.name if hasattr(src_model,'name') else 'src_model'
  print('Cloning the model '+src_model_name)
  input_model = keras.models.clone_model(src_model)
  print('Popping the last layer')
  for i in range(n_layers_to_pop):
    input_model.layers.pop()
  input_model_output = input_model.layers[-1].output
  added_layer = layer_dict['type'](layer_dict['kernels'],activation=layer_dict['activation'])(input_model_output)
  last_layer=Dense(10,activation='softmax')(added_layer)
  print('Creating new model by adding a penultimate Dense layer')
  modified_model = Model(input_model.inputs,last_layer,name='Modified '+src_model_name)
  print('Copying weights from ',src_model_name,'to',modified_model.name)
  if copy_weights:
    #if 1<0:
    #  modified_model.set_weights(src_model.get_weights())
    #else:
    for i in range(len(input_model.layers)):
        print('\rCopying weights of layer:',i+1,end='')
        try:
          modified_model.layers[i].set_weights(input_model.layers[i].get_weights())
        except Exception as e:
          print('\nError copying weights at layer ',i+1)
    #modified_model.layers[-1].set_weights(src_model.layers[-1].get_weights())
  return modified_model

In [30]:
model = DenseNet(depth=7,nb_dense_block=2)

#new_model = modified_model(model,True)
new_model = add_penultimaye_layer(model,{'type':Dense,'kernels':1024,'activation':'linear'},copy_weights=True)

Cloning the model densenet
Popping the last layer
Creating new model by adding a penultimate Dense layer
Copying weights from  densenet to Modified densenet
Copying weights of layer: 16

In [0]:
new_model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True))

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
initial_conv2D (Conv2D)         (None, 32, 32, 24)   648         input_6[0][0]                    
__________________________________________________________________________________________________
dense_0_0_bn (BatchNormalizatio (None, 32, 32, 24)   96          initial_conv2D[0][0]             
__________________________________________________________________________________________________
activation_21 (Activation)      (None, 32, 32, 24)   0           dense_0_0_bn[0][0]               
__________________________________________________________________________________________________
dense_0_0_